In [1]:
# Load dataset

import json
input_file = "dataset/org_with_cefr_labels/us_RL.jsonl"
dataset = [json.loads(line.strip()) for line in open(input_file)]
dataset = dataset[:2]


In [ ]:
# Augment dataset with rollouts

import json
from vllm import SamplingParams
from inference import TextGenerator

model_name = "google/gemma-3-1b-it"
sampling_params_list = [
    SamplingParams(temperature=0.3, top_p=0.85, n=2, max_tokens=1024),
    SamplingParams(temperature=0.7, top_p=0.9, n=2, max_tokens=1024),
]

text_generator = TextGenerator(model_name, sampling_params_list)

text_generator.generate_dataset(dataset)

with open("tmp_dataset_with_predictions.jsonl", "w") as fp:
    for instance in datsaet:
        fp.write(json.dumps(instance) + "\n")


In [5]:
# Augment dataset with rewards

import torch
from transformers import pipeline
from prompt import CEFR_LABELS


# Load intermediate output if there are any issues fails.
# dataset = []
# with open("tmp_dataset_with_predictions.jsonl") as fp:
#     for line in fp:
#         dataset.append(json.loads(line.strip()))

all_texts = []
all_references = []
metadata = [] # To keep track of (item_index, label, candidate_index)

for i, item in enumerate(dataset):
    for label in cefr_labels:
        preds = item["predictions"][label]
        # Ensure it's a list even if it's a single string by mistake
        if isinstance(preds, str): preds = [preds] 
        
        for k, cand_text in enumerate(preds):
            all_texts.append(cand_text)
            all_references.append(item["summary"])
            metadata.append((i, label, k))

# 2. Bulk CEFR Scoring
print(f"Scoring {len(all_texts)} total candidates...")
# Feeding a list to the pipeline is much faster
cefr_probs_flat = list(tqdm(classifier(all_texts), total=len(all_texts)))

# 3. Bulk BERTScore
_, _, f1_flat = score(
    all_texts, 
    all_references, 
    model_type="microsoft/deberta-xlarge-mnli", 
    lang="en", 
    batch_size=32, 
    device="cuda:0"
)
bert_scores_flat = f1_flat.tolist()

# 4. Re-shape back into the nested structure
# We initialize a structure to mirror the 'predictions' list structure
for i in range(len(dataset)):
    dataset[i]["evaluation"] = {label: [] for label in cefr_labels}

for idx, (item_idx, label, cand_idx) in enumerate(metadata):
    # Extract CEFR prob for the specific label we requested
    probs_dict = {p["label"]: p["score"] for p in cefr_probs_flat[idx]}
    c_score = probs_dict.get(label, 0.0)
    
    # Pack the scores
    eval_entry = {
        "cefr_prob": c_score,
        "bert_score": bert_scores_flat[idx]
    }
    
    # Append to the list corresponding to that label
    dataset[item_idx]["evaluation"][label].append(eval_entry)



Device set to use cuda:0


In [ ]:
# Construct DPO dataset
